In [1]:
import sys
# setting path
sys.path.append('../RSA')
# importing
import Shor_library
from qiskit import IBMQ
import random
from math import gcd
import math
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer, transpile
from qiskit.aqua import QuantumInstance

ModuleNotFoundError: No module named 'qiskit'

In [ ]:
SEMIPRIMES=[15,21,33,65,143]

In [ ]:
def circuit_builder(N,a,sequential,method):
    n = math.ceil(math.log(N,2))
    if method=='Basic':
        """initialize the registers and the circuit"""
        aux = QuantumRegister(n+2)
        if sequential=='False':
            up_reg = QuantumRegister(2*n)
        else:
            up_reg = QuantumRegister(1)
            c_aux = ClassicalRegister(1)
        down_reg = QuantumRegister(n)
        up_classic = ClassicalRegister(2*n)
        if sequential=='False':
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic)
        else:
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic,c_aux)
        """Create the circuit"""
        if sequential=='False':
            """ Initialize down register to 1 and create maximal superposition in top register """
            circuit.h(up_reg)
            circuit.x(down_reg[0])
            """ Apply the multiplication gates as showed in the report in order to create the exponentiation """
            for i in range(0, 2*n):
                Shor_library.Ua_basic(circuit, up_reg[i], down_reg, aux, int(pow(a, pow(2, i))), N, n)
            """ Apply inverse QFT """
            Shor_library.create_inverse_QFT(circuit, up_reg, 2*n ,1)
            """ Measure the top qubits, to get x value"""
            circuit.measure(up_reg,up_classic)
        else:
            """ Initialize down register to 1"""
            circuit.x(down_reg[0])
            """ Cycle to create the Sequential QFT, measuring qubits and applying the right gates according to measurements """
            for i in range(0, 2*n):
                """reset the top qubit to 0 if the previous measurement was 1"""
                circuit.x(up_reg).c_if(c_aux, 1)
                circuit.h(up_reg)
                Shor_library.Ua_basic(circuit, up_reg[0], down_reg, aux, a**(2**(2*n-1-i)), N, n)
                """cycle through all possible values of the classical register and apply the corresponding conditional phase shift"""
                for j in range(0, 2**i):
                    """the phase shift is applied if the value of the classical register matches j exactly"""
                    circuit.p(Shor_library.getAngle(j, i), up_reg[0]).c_if(up_classic, j)
                circuit.h(up_reg)
                circuit.measure(up_reg[0], up_classic[i])
                circuit.measure(up_reg[0], c_aux[0])
    else:
        """Case with inplace_adder"""
        if sequential=='False':
            aux = QuantumRegister(n+1)                          
            up_reg = QuantumRegister(2*n)
            down_reg = QuantumRegister(n)
            up_classic = ClassicalRegister(2*n)
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic)
            circuit.h(up_reg)
            circuit.x(down_reg[0])
            for i in range(0, 2*n):
                Shor_library.Ua_inplace(circuit, up_reg[i:i+1], down_reg, aux, int(pow(a, pow(2, i))), N, n)
            Shor_library.create_inverse_QFT(circuit, up_reg, 2*n ,1)
            circuit.measure(up_reg,up_classic)
        else:
            aux = QuantumRegister(n+1)                          
            up_reg = QuantumRegister(1)
            down_reg = QuantumRegister(n)
            up_classic = ClassicalRegister(2*n)
            c_aux = ClassicalRegister(1)
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic,c_aux)
            circuit.x(down_reg[0])
            for i in range(0, 2*n):
                """reset the top qubit to 0 if the previous measurement was 1"""
                circuit.x(up_reg).c_if(c_aux, 1)
                circuit.h(up_reg)
                Shor_library.Ua_inplace(circuit, up_reg[0:1], down_reg, aux, a**(2**(2*n-1-i)), N, n)
                """cycle through all possible values of the classical register and apply the corresponding conditional phase shift"""
                for j in range(0, 2**i):
                    """the phase shift is applied if the value of the classical register matches j exactly"""
                    circuit.p(Shor_library.getAngle(j, i), up_reg[0]).c_if(up_classic, j)
                circuit.h(up_reg)
                circuit.measure(up_reg[0], up_classic[i])
                circuit.measure(up_reg[0], c_aux[0])
    return circuit

In [ ]:
print('Tests for depth of the circuits')
a=2
for N in SEMIPRIMES:
    print(N)
    print('    Normal QFT, Basic')
    d_vec=[]
    #for a in range(2,N-1):
    #    if gcd(a,N)==1:
    circuit = circuit_builder(N,a,'False','Basic')
    dec1 = circuit.decompose()
    dec2 = dec1.decompose()
    dec3 = dec2.decompose()
    dec4 = dec3.decompose()
    d_vec.append(dec1.depth())
    d_vec.append(dec2.depth())
    d_vec.append(dec3.depth())
    d_vec.append(dec4.depth())
    print(d_vec)
    print('----------------------------------')

    print('    Normal QFT, InPlace')
    d_vec=[]
    #for a in range(2,N-1):
    #    if gcd(a,N)==1:
    circuit = circuit_builder(N,a,'False','InPlace')
    dec1 = circuit.decompose()
    dec2 = dec1.decompose()
    dec3 = dec2.decompose()
    dec4 = dec3.decompose()
    d_vec.append(dec1.depth())
    d_vec.append(dec2.depth())
    d_vec.append(dec3.depth())
    d_vec.append(dec4.depth())
    print(d_vec)
    print('----------------------------------')

    print('    Sequential QFT, Basic')
    d_vec=[]
    #for a in range(2,N-1):
    #    if gcd(a,N)==1:
    circuit = circuit_builder(N,a,'True','Basic')
    dec1 = circuit.decompose()
    dec2 = dec1.decompose()
    dec3 = dec2.decompose()
    dec4 = dec3.decompose()
    d_vec.append(dec1.depth())
    d_vec.append(dec2.depth())
    d_vec.append(dec3.depth())
    d_vec.append(dec4.depth())
    print(d_vec)
    print('----------------------------------')

    print('    Sequential QFT, InPlace')
    d_vec=[]
    #for a in range(2,N-1):
    #    if gcd(a,N)==1:
    circuit = circuit_builder(N,a,'True','InPlace')
    dec1 = circuit.decompose()
    dec2 = dec1.decompose()
    dec3 = dec2.decompose()
    dec4 = dec3.decompose()
    d_vec.append(dec1.depth())
    d_vec.append(dec2.depth())
    d_vec.append(dec3.depth())
    d_vec.append(dec4.depth())
    print(d_vec)
    print('--------------------------------------------------------')